In [4]:
import pandas as pd
import requests
import json 
import folium 
import os
from datetime import datetime, timedelta

In [70]:
def sismo(url):
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=20000)).strftime('%Y-%m-%dT%H:%M:%S')

    # Establecer los demás parámetros de solicitud
    params = {"format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 0,
            "minlatitude": -18.521, "maxlatitude": 0.132, "minlongitude": -84.419, "maxlongitude": -68.599, "limit": 20000}    
    data = requests.get(url, params = params)
    data = json.loads(data.text)
    return data

url = r"https://earthquake.usgs.gov/fdsnws/event/1/query?"
dic_sismo = sismo(url)
count=0
for i in (dic_sismo["features"]):
    if i["properties"]['cdi']:
        count += 1
# Convertir los datos a un dataframe
df_sismo = pd.json_normalize(dic_sismo['features'])

In [71]:
df_sismo.head()

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,us6000kcsh,4.3,"11 km WSW of Pucara, Peru",1684408298865,1684411380040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",31.0,2.544,1.32,126.0,mb,earthquake,"M 4.3 - 11 km WSW of Pucara, Peru",Point,"[-71.5665, -16.0072, 131.861]"
1,Feature,us6000kcfu,4.0,"9 km WNW of Ayo, Peru",1684289721181,1684377795040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",17.0,3.320,0.66,154.0,mb,earthquake,"M 4.0 - 9 km WNW of Ayo, Peru",Point,"[-72.3568, -15.6528, 8.284]"
2,Feature,us6000kbqc,4.5,"6 km NW of Paján, Ecuador",1684004377115,1684005752040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",69.0,1.301,0.90,150.0,mb,earthquake,"M 4.5 - 6 km NW of Paján, Ecuador",Point,"[-80.4691, -1.5056, 43.612]"
3,Feature,us6000kbad,4.3,"31 km NE of Palca, Peru",1683854607800,1683856356040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",14.0,0.231,0.45,115.0,mb,earthquake,"M 4.3 - 31 km NE of Palca, Peru",Point,"[-69.721, -17.6075, 163.176]"
4,Feature,us6000kaam,5.2,"8 km W of Azogues, Ecuador",1683507115846,1683606904934,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,23.0,...,",dyfi,moment-tensor,origin,phase-data,",113.0,1.250,0.58,39.0,mww,earthquake,"M 5.2 - 8 km W of Azogues, Ecuador",Point,"[-78.9266, -2.7512, 89.603]"


In [72]:
df_sismo.columns

Index(['type', 'id', 'properties.mag', 'properties.place', 'properties.time',
       'properties.updated', 'properties.tz', 'properties.url',
       'properties.detail', 'properties.felt', 'properties.cdi',
       'properties.mmi', 'properties.alert', 'properties.status',
       'properties.tsunami', 'properties.sig', 'properties.net',
       'properties.code', 'properties.ids', 'properties.sources',
       'properties.types', 'properties.nst', 'properties.dmin',
       'properties.rms', 'properties.gap', 'properties.magType',
       'properties.type', 'properties.title', 'geometry.type',
       'geometry.coordinates'],
      dtype='object')

In [73]:
df_sismo.drop(columns=['type','properties.alert','properties.tz','properties.code','properties.url','properties.detail','properties.net'
                       ,'properties.status','properties.types','properties.sources','properties.magType','properties.type','geometry.type',
                       'properties.title'], axis=1, inplace=True)

In [74]:
df_sismo['Pais'] = df_sismo['properties.place'].str.split(',').str[1].str.strip()

df_sismo['properties.place'] = df_sismo['properties.place'].str.split(',', expand=True)[0]
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,Pais
0,us6000kcsh,4.3,11 km WSW of Pucara,1684408298865,1684411380040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,"[-71.5665, -16.0072, 131.861]",Peru


In [75]:
df_sismo["properties.time"] = pd.to_datetime(df_sismo["properties.time"], unit="ms") 
df_sismo["properties.updated"] = pd.to_datetime(df_sismo["properties.updated"], unit="ms")

In [76]:
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,Pais
0,us6000kcsh,4.3,11 km WSW of Pucara,2023-05-18 11:11:38.865,2023-05-18 12:03:00.040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,"[-71.5665, -16.0072, 131.861]",Peru


In [77]:
#Divido la columna geometry coordinates
df_sismo[['Longitud', 'Latitud', 'Profundidad']] = df_sismo['geometry.coordinates'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))

In [78]:
df_sismo["Longitud"] = df_sismo["Longitud"].astype(float) 
df_sismo["Latitud"] = df_sismo["Latitud"].astype(float) 
df_sismo["Profundidad"] = df_sismo["Profundidad"].astype(float)

In [79]:
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,Pais,Longitud,Latitud,Profundidad
0,us6000kcsh,4.3,11 km WSW of Pucara,2023-05-18 11:11:38.865,2023-05-18 12:03:00.040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,"[-71.5665, -16.0072, 131.861]",Peru,-71.5665,-16.0072,131.861


In [80]:
df_sismo.drop("geometry.coordinates", axis=1, inplace=True) 

In [81]:
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,Pais,Longitud,Latitud,Profundidad
0,us6000kcsh,4.3,11 km WSW of Pucara,2023-05-18 11:11:38.865,2023-05-18 12:03:00.040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,Peru,-71.5665,-16.0072,131.861


In [82]:
df_sismo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9960 entries, 0 to 9959
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  9960 non-null   object        
 1   properties.mag      9960 non-null   float64       
 2   properties.place    9942 non-null   object        
 3   properties.time     9960 non-null   datetime64[ns]
 4   properties.updated  9960 non-null   datetime64[ns]
 5   properties.felt     1490 non-null   float64       
 6   properties.cdi      1490 non-null   float64       
 7   properties.mmi      485 non-null    float64       
 8   properties.tsunami  9960 non-null   int64         
 9   properties.sig      9960 non-null   int64         
 10  properties.ids      9960 non-null   object        
 11  properties.nst      3875 non-null   float64       
 12  properties.dmin     2799 non-null   float64       
 13  properties.rms      8869 non-null   float64     

In [83]:
df_sismo = df_sismo.rename(columns={'properties.mag':      'Magnituud',   
                                    'properties.place':    'Place',
                                    'properties.time':     'Fecha',  
                                    'properties.updated':  'Ultimo registro', 
                                    'properties.felt':     'Felt',
                                    'properties.cdi':      'cdi',
                                    'properties.mmi':      'mmi', 
                                    'properties.tsunami':  'Posibilidad tsunami',
                                    'properties.sig':      'Importancia del evento',
                                    'properties.ids':      'Ids',
                                    'properties.nst':      'nst',
                                    'properties.dmin':     'Distancia Horizontal Epicentro',
                                    'properties.rms':      'RMS',
                                    'properties.gap':      'Brecha Azimutal'})
df_sismo.head(1)

,id,Magnituud,Place,Fecha,Ultimo registro,Felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,Ids,nst,Distancia Horizontal Epicentro,RMS,Brecha Azimutal,Pais,Longitud,Latitud,Profundidad
0,us6000kcsh,4.3,11 km WSW of Pucara,2023-05-18 11:11:38.865,2023-05-18 12:03:00.040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,Peru,-71.5665,-16.0072,131.861


In [84]:
df_sismo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9960 entries, 0 to 9959
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              9960 non-null   object        
 1   Magnituud                       9960 non-null   float64       
 2   Place                           9942 non-null   object        
 3   Fecha                           9960 non-null   datetime64[ns]
 4   Ultimo registro                 9960 non-null   datetime64[ns]
 5   Felt                            1490 non-null   float64       
 6   cdi                             1490 non-null   float64       
 7   mmi                             485 non-null    float64       
 8   Posibilidad tsunami             9960 non-null   int64         
 9   Importancia del evento          9960 non-null   int64         
 10  Ids                             9960 non-null   object        
 11  nst 

In [5]:
url_USGS = r"https://earthquake.usgs.gov/fdsnws/event/1/query?"

filename_json           = 'Peru_data.json'
folder_path_SE_JSON     = '../JSON/SIN_ETL'
folder_path_CE_JSON     = '../JSON/CON_ETL'

filename_csv            = 'Peru_data.csv'
folder_path_SE_CSV      = '../DASHBOARD/CSV_ORIGINAL'
folder_path_CE_CSV      = '../DASHBOARD/CSV_TRANSFORMADOS'



def sismo_Peru_json(url_USGS,filename_json,folder_path_SE_JSON):
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=20000)).strftime('%Y-%m-%dT%H:%M:%S')
    
    # Establecer los demás parámetros de solicitud
    params = {"format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 2.5,
                "minlatitude": -18.521, "maxlatitude": 0.132, "minlongitude": -84.419, "maxlongitude": -68.599, "limit": 20000}
    
    # Realizar la solicitud a la API de USGS
    response = requests.get(url_USGS, params=params)
    data = response.json()
    
    # Obtener solo las características (features)
    features = data['features']
    
    # Convertir las características a un DataFrame
    df_sismo = pd.json_normalize(features)
    
    # Guardar el resultado en un archivo JSON        
    file_path = os.path.join(folder_path_SE_JSON, filename_json)
    df_sismo.to_json(file_path, orient='records')
    
    return df_sismo


In [6]:
sismo_Peru_json(url_USGS,filename_json,folder_path_SE_JSON)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,us6000kcsh,4.30,"11 km WSW of Pucara, Peru",1684408298865,1684411380040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",31.0,2.544,1.32,126.0,mb,earthquake,"M 4.3 - 11 km WSW of Pucara, Peru",Point,"[-71.5665, -16.0072, 131.861]"
1,Feature,us6000kcfu,4.00,"9 km WNW of Ayo, Peru",1684289721181,1684377795040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",17.0,3.320,0.66,154.0,mb,earthquake,"M 4.0 - 9 km WNW of Ayo, Peru",Point,"[-72.3568, -15.6528, 8.284]"
2,Feature,us6000kbqc,4.50,"6 km NW of Paján, Ecuador",1684004377115,1684005752040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",69.0,1.301,0.90,150.0,mb,earthquake,"M 4.5 - 6 km NW of Paján, Ecuador",Point,"[-80.4691, -1.5056, 43.612]"
3,Feature,us6000kbcv,4.50,"36 km NW of El Cardo, Peru",1683882523176,1684484511040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",90.0,3.204,0.46,140.0,mb,earthquake,"M 4.5 - 36 km NW of El Cardo, Peru",Point,"[-72.9886, -16.3897, 67.82]"
4,Feature,us6000kbad,4.30,"31 km NE of Palca, Peru",1683854607800,1683856356040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",14.0,0.231,0.45,115.0,mb,earthquake,"M 4.3 - 31 km NE of Palca, Peru",Point,"[-69.721, -17.6075, 163.176]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9752,Feature,iscgem816873,5.54,"31 km E of Honoria, Peru",-41383479820,1651011649512,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,"M 5.5 - 31 km E of Honoria, Peru",Point,"[-74.425, -8.73, 146.8]"
9753,Feature,iscgem816821,5.28,"57 km N of Segunda Jerusalén - Azunguillo, Peru",-41494684740,1651011636279,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,M 5.3 - 57 km N of Segunda Jerusalén - Azungui...,Point,"[-77.219, -5.473, 24.3]"
9754,Feature,iscgem816750,5.53,"56 km ESE of Sechura, Peru",-41616616100,1651011629274,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,"M 5.5 - 56 km ESE of Sechura, Peru",Point,"[-80.391, -5.838, 66.2]"
9755,Feature,iscgem818377,5.19,"35 km NNW of Segunda Jerusalén - Azunguillo, Peru",-42468044130,1651011937393,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,M 5.2 - 35 km NNW of Segunda Jerusalén - Azung...,Point,"[-77.357, -5.677, 20]"


In [7]:
def sismo_Peru_json_etl(url_USGS, filename_json,folder_path_CE_JSON):
    
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=20000)).strftime('%Y-%m-%dT%H:%M:%S')
    
    # Establecer los demás parámetros de solicitud
    params = {"format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 2.5,
                "minlatitude": -18.521, "maxlatitude": 0.132, "minlongitude": -84.419, "maxlongitude": -68.599, "limit": 20000}
    
    # Realizar la solicitud a la API de USGS
    response = requests.get(url_USGS, params=params)
    data = response.json()
    
    # Obtener solo las características (features)
    features = data['features']
    
    # Convertir las características a un DataFrame
    df_sismo = pd.json_normalize(features)
    
    df_sismo.drop(columns=['type','properties.alert','properties.tz','properties.code',
                           'properties.url','properties.detail','properties.net',
                           'properties.status','properties.types','properties.sources',
                           'properties.magType','properties.type','geometry.type','properties.title'], axis=1, inplace=True)
    
    df_sismo['Pais'] = df_sismo['properties.place'].str.split(',').str[1].str.strip()
    df_sismo['properties.place'] = df_sismo['properties.place'].str.split(',', expand=True)[0]
    
    # Transformar las columnas de tiempo
    df_sismo["properties.time"] = pd.to_datetime(df_sismo["properties.time"], unit="ms")
    df_sismo["properties.updated"] = pd.to_datetime(df_sismo["properties.updated"], unit="ms")
            
    # Dividir la columna geometry.coordinates en columnas separadas
    df_sismo[["Longitud", "Latitud", "Profundidad"]] = pd.DataFrame(df_sismo["geometry.coordinates"].tolist())
    
    df_sismo.drop("geometry.coordinates", axis=1, inplace=True) 
       
    df_sismo = df_sismo.rename(columns={'properties.mag':      'Magnituud',
                                        'properties.place':    'Place',
                                        'properties.time':     'Fecha',
                                        'properties.updated':  'Ultimo registro',
                                        'properties.felt':     'Felt',
                                        'properties.cdi':      'cdi',
                                        'properties.mmi':      'mmi',
                                        'properties.tsunami':  'Posibilidad tsunami',
                                        'properties.sig':      'Importancia del evento',
                                        'properties.ids':      'Ids',
                                        'properties.nst':      'nst',
                                        'properties.dmin':     'Distancia Horizontal Epicentro',
                                        'properties.rms':      'RMS',
                                        'properties.gap':      'Brecha Azimutal'})
        
    df_sismo["Longitud"] = df_sismo["Longitud"].astype(float) 
    df_sismo["Latitud"] = df_sismo["Latitud"].astype(float) 
    df_sismo["Profundidad"] = df_sismo["Profundidad"].astype(float)   
    
    file_path = os.path.join(folder_path_CE_JSON, filename_json)
    df_sismo.to_json(file_path, orient='records')
    
    # Devolver el DataFrame
    return df_sismo


In [8]:
sismo_Peru_json_etl(url_USGS, filename_json,folder_path_CE_JSON)

,id,Magnituud,Place,Fecha,Ultimo registro,Felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,Ids,nst,Distancia Horizontal Epicentro,RMS,Brecha Azimutal,Pais,Longitud,Latitud,Profundidad
0,us6000kcsh,4.30,11 km WSW of Pucara,2023-05-18 11:11:38.865,2023-05-18 12:03:00.040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,Peru,-71.5665,-16.0072,131.861
1,us6000kcfu,4.00,9 km WNW of Ayo,2023-05-17 02:15:21.181,2023-05-18 02:43:15.040,NaN,NaN,NaN,0,246,",us6000kcfu,",17.0,3.320,0.66,154.0,Peru,-72.3568,-15.6528,8.284
2,us6000kbqc,4.50,6 km NW of Paján,2023-05-13 18:59:37.115,2023-05-13 19:22:32.040,NaN,NaN,NaN,0,312,",us6000kbqc,",69.0,1.301,0.90,150.0,Ecuador,-80.4691,-1.5056,43.612
3,us6000kbcv,4.50,36 km NW of El Cardo,2023-05-12 09:08:43.176,2023-05-19 08:21:51.040,NaN,NaN,NaN,0,312,",us6000kbcv,",90.0,3.204,0.46,140.0,Peru,-72.9886,-16.3897,67.820
4,us6000kbad,4.30,31 km NE of Palca,2023-05-12 01:23:27.800,2023-05-12 01:52:36.040,NaN,NaN,NaN,0,284,",us6000kbad,",14.0,0.231,0.45,115.0,Peru,-69.7210,-17.6075,163.176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9752,iscgem816873,5.54,31 km E of Honoria,1968-09-09 00:35:20.180,2022-04-26 22:20:49.512,NaN,NaN,NaN,0,472,",iscgem816873,",NaN,NaN,NaN,NaN,Peru,-74.4250,-8.7300,146.800
9753,iscgem816821,5.28,57 km N of Segunda Jerusalén - Azunguillo,1968-09-07 17:41:55.260,2022-04-26 22:20:36.279,NaN,NaN,NaN,0,429,",iscgem816821,",NaN,NaN,NaN,NaN,Peru,-77.2190,-5.4730,24.300
9754,iscgem816750,5.53,56 km ESE of Sechura,1968-09-06 07:49:43.900,2022-04-26 22:20:29.274,NaN,NaN,NaN,0,470,",iscgem816750,",NaN,NaN,NaN,NaN,Peru,-80.3910,-5.8380,66.200
9755,iscgem818377,5.19,35 km NNW of Segunda Jerusalén - Azunguillo,1968-08-27 11:19:15.870,2022-04-26 22:25:37.393,NaN,NaN,NaN,0,414,",iscgem818377,",NaN,NaN,NaN,NaN,Peru,-77.3570,-5.6770,20.000


In [ ]:
def sismo_Peru_CSV(url_USGS,folder_path_SE_CSV, filename_csv):
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=20000)).strftime('%Y-%m-%dT%H:%M:%S')
    
    # Establecer los demás parámetros de solicitud
    params = {"format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 2.5,
                "minlatitude": -18.521, "maxlatitude": 0.132, "minlongitude": -84.419, "maxlongitude": -68.599, "limit": 20000}
    
    # Realizar la solicitud a la API de USGS
    response = requests.get(url_USGS, params=params)
    data = response.json()
    
    # Obtener solo las características (features)
    features = data['features']
    
    # Convertir las características a un DataFrame
    df_sismo = pd.json_normalize(features)
    
    # Guardar el resultado en un archivo JSON        
    file_path = os.path.join(folder_path_SE_CSV, filename_csv)
    df_sismo.to_csv(file_path, index=False)
    
    return df_sismo


In [ ]:



def sismo_Peru_CSV_ETL(url_USGS, filename_csv,folder_path_CE_CSV):
    
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=20000)).strftime('%Y-%m-%dT%H:%M:%S')
    
    # Establecer los demás parámetros de solicitud
    params = {"format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 2.5,
                "minlatitude": -18.521, "maxlatitude": 0.132, "minlongitude": -84.419, "maxlongitude": -68.599, "limit": 20000}
    
    # Realizar la solicitud a la API de USGS
    response = requests.get(url_USGS, params=params)
    data = response.json()
    
    # Obtener solo las características (features)
    features = data['features']
    
    # Convertir las características a un DataFrame
    df_sismo = pd.json_normalize(features)
    
    df_sismo.drop(columns=['type','properties.alert','properties.tz','properties.code','properties.url','properties.detail','properties.net'
                       ,'properties.status','properties.types','properties.sources','properties.magType','properties.type','geometry.type',
                       'properties.title'], axis=1, inplace=True)
    
    df_sismo['Pais'] = df_sismo['properties.place'].str.split(',').str[1].str.strip()
    df_sismo['properties.place'] = df_sismo['properties.place'].str.split(',', expand=True)[0]
    
    # Transformar las columnas de tiempo
    df_sismo["properties.time"] = pd.to_datetime(df_sismo["properties.time"], unit="ms")
    df_sismo["properties.updated"] = pd.to_datetime(df_sismo["properties.updated"], unit="ms")
    
        
    # Dividir la columna geometry.coordinates en columnas separadas
    df_sismo[["Longitud", "Latitud", "Profundidad"]] = pd.DataFrame(df_sismo["geometry.coordinates"].tolist())
    
    df_sismo.drop("geometry.coordinates", axis=1, inplace=True) 
       
    df_sismo = df_sismo.rename(columns={'properties.mag':      'Magnituud',   
                                        'properties.place':    'Place',
                                        'properties.time':     'Fecha',  
                                        'properties.updated':  'Ultimo registro', 
                                        'properties.felt':     'Felt',
                                        'properties.cdi':      'cdi',
                                        'properties.mmi':      'mmi', 
                                        'properties.tsunami':  'Posibilidad tsunami',
                                        'properties.sig':      'Importancia del evento',
                                        'properties.ids':      'Ids',
                                        'properties.nst':      'nst',
                                        'properties.dmin':     'Distancia Horizontal Epicentro',
                                        'properties.rms':      'RMS',
                                        'properties.gap':      'Brecha Azimutal'})
     
    df_sismo["Longitud"] = df_sismo["Longitud"].astype(float) 
    df_sismo["Latitud"] = df_sismo["Latitud"].astype(float) 
    df_sismo["Profundidad"] = df_sismo["Profundidad"].astype(float)   
    
    file_path = os.path.join(folder_path_CE_CSV, filename_csv)
    df_sismo.to_csv(file_path, index=False)

    
    # Devolver el DataFrame
    return df_sismo

In [85]:
#folder_path = "../DASHBOARD/CSV_TRANSFORMADOS"
#file_name = "Datos_Peru_USGS.csv"
#file_path = os.path.join(folder_path, file_name)
#
#df_sismo.to_csv(file_path, index=False)